In [1]:
# ********************************************************************************************************************************  
# WARNING DATA ARE IN UTC! PLEASE ADJUST TIME STAMPS FOR PROJECT TIME ZONE 
# ********************************************************************************************************************************  
# Hub height (m AGL)       : 100m 
# Path to ERA5 files       : /local/ERA5_ASII_South_23084_merged_windspeed.nc 
# Station name             : ASII_South 
# Station latitude         : -14.0855 
# Station longitude        : -42.6109 
# Model latitude/longitude : [-14.09999999999999,-42.60000000000002] [-13.8,-42.60000000000002] [-13.8,-42.30000000000001] [-14.09999999999999,-42.30000000000001] [-14.39999999999999,-42.30000000000001] [-14.39999999999999,-42.60000000000002] [-14.39999999999999,-42.90000000000003] [-14.09999999999999,-42.90000000000003] [-13.8,-42.90000000000003] 
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  
# Date           Time(UTC)       c_ws       c_wd         n_ws       n_wd        ne_ws      ne_wd         e_ws       e_wd        se_ws      se_wd         s_ws       s_wd        sw_ws      sw_wd         w_ws       w_wd        nw_ws      nw_wd  
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------  


import pandas as pd, numpy as np
from pathlib import Path
root = Path(r'/home/diogo/Downloads/ERA5')

def get_data(root):
    for in_path in root.rglob('*.txt'):
        with open(in_path) as in_file:
            if not in_file.readline().startswith('*'): 
                out_path = Path(r'/home/diogo/Downloads/ERA5-useless_header') / in_path.name
                in_path.rename(out_path)
                continue
            [in_file.readline() for i in range(5)]
            lat, lon = [in_file.readline().split(':')[-1].strip() for _ in range(2)]
            yield [str(in_path), in_path.parts[-1].replace('_ERA5', '').replace('.txt', ''), lat, lon]
df = pd.DataFrame(get_data(root), columns=['path', 'name', 'lat', 'lon'])
df[['lat', 'lon', 'name']].to_csv(root / 'coordinates.csv', index=False)

In [36]:
import pandas as pd, numpy as np, pyperclip
from pathlib import Path
in_path = Path(r'/home/diogo/Jupyter/tdef/result.csv')
df = pd.read_csv(in_path,sep='\s+', engine='python').set_index('path')
df = df.sort_values('MAPE')
# \hline
# Modelo&ME&RMSE&MAE&MPE&MAPE \\
# \hline
# GARCH(1,1)&-0.04246845&0.06505767&0.05117909&-2.449632&2.592525 \\

out_path = in_path.parent / f'{in_path.stem}-table.txt'
with open(out_path, 'w') as out_file:
    out_file.write('\\begin{center}\n')
    out_file.write('\\begin{tabular}{ |c|c|c|c|c|c| } \n')
    out_file.write('\\hline\n')
    out_file.write('Local&ME&RMSE&MAE&MPE&MAPE\\\\\n')
    out_file.write('\\hline\n')    
    for path, row in df.tail(10).iterrows():
        numbers = '&'.join([str(round(r,2)) for r in row])
        path = path.replace('_', r'\_')
        out_file.write(f'{path}&{numbers}\\\\\n')
        out_file.write('\\hline\n')
    out_file.write('\\end{tabular}\n')
    out_file.write('\\end{center}\n')
pyperclip.copy(out_path.read_text())

In [26]:
import pandas as pd, numpy as np, pyperclip
from pathlib import Path
in_path = Path(r'/home/diogo/Jupyter/tdef/thesis/tdef.tex')
out_path = Path(r'/home/diogo/Jupyter/tdef/thesis/slides.txt')

def get_data(in_path):
    with open(in_path) as in_file:
        begin_figure, end_figure = False, False
        image = None
        for line in in_file:
            if line.strip().startswith('%'): continue
            if 'begin{figure}' in line:
                begin_figure = True
            if begin_figure:
                if 'includegraphics' in line:
                    image = line.strip().split('{')[-1].replace('}', '').replace('\n', '')
                if 'caption' in line:
                    caption = line.strip().split('{')[-1].replace('}', '').replace('\n', '')
                if 'end{figure}' in line:
                    yield [image, caption]

with open(out_path, 'w') as out_file:
    df = pd.DataFrame(get_data(path), columns=['image', 'caption']).set_index('image')
    for image, caption in df.itertuples():
        out_file.write(r'\begin{frame}'+'\n')
        out_file.write('\t'+f'\\frametitle{{{caption}}}'+'\n')
        out_file.write('\t'+r'\begin{figure}'+'\n')
        out_file.write('\t\t'+r'\centering'+'\n')
        out_file.write('\t\t'+f'\includegraphics[width=\\textwidth]{{{image}}}'+'\n')
        out_file.write('\t\t'+f'\caption{{{caption}}}'+'\n')
        out_file.write('\t'+r'\end{figure}'+'\n')
        out_file.write(r'\end{frame}'+'\n\n')
pyperclip.copy(out_path.read_text())